In [1]:
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import scipy.io
import scipy.misc
import numpy as np
import pandas as pd
import PIL
from keras import backend as K
from keras.layers import Input, Lambda, Conv2D
from keras.models import load_model, Model




Using TensorFlow backend.


In [0]:
def yolo_filter_boxes(box_confidence, boxes, box_class_probs, threshold = 0.6):
  box_scores = np.multiply(box_confidence,box_class_probs)
  box_classes = K.argmax(box_scores, axis = -1)
  box_class_scores = K.max(box_scores, axis = -1)
  filtering_mask = K.greater_equal(box_class_scores, threshold)
  scores = tf.boolean_mask(box_class_scores, filtering_mask)
  boxes = tf.boolean_mask(boxes, filtering_mask)
  classes = tf.boolean_mask(box_classes, filtering_mask)
  print(box_scores.shape)
  print(box_classes.shape)
  print(box_class_scores.shape)
  print(filtering_mask.shape)
  return scores, boxes, classes

In [3]:
with tf.Session() as test_a:
  box_confidence = tf.random_normal([19,19,5,1], mean = 1, stddev = 4, seed =1)
  boxes = tf.random_normal([19,19,5,4], mean = 1, stddev = 4, seed =1)
  box_class_probs = tf.random_normal([19,19,5,80], mean = 1, stddev = 4, seed =1)
  scores, boxes, classes = yolo_filter_boxes(box_confidence, boxes, box_class_probs, threshold = 0.5) 
  print(scores[2].eval())
  print(boxes[2].eval())
  print(classes[2].eval())
  print(scores.shape)
  print(boxes.shape)
  print(classes.shape)

W0622 14:20:24.241814 139694429996928 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/array_ops.py:1354: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


(19, 19, 5, 80)
(19, 19, 5)
(19, 19, 5)
(19, 19, 5)
10.750582
[ 8.426533   3.2713668 -0.5313436 -4.9413733]
7
(?,)
(?, 4)
(?,)


In [0]:
#doing intersection over union
def iou(box1,box2):
  xi1 = max(box1[0],box2[0])
  yi1 = max(box1[1],box2[1])
  xi2 = min(box1[2],box2[2])
  yi2 = min(box1[3],box2[3])
  inter_area = (xi2-xi1)*(yi2-yi1)
  box1_area = (box1[3] - box1[1])*(box1[2]- box1[0])
  box2_area = (box2[3] - box2[1])*(box2[2]- box2[0])
  union_area = (box1_area + box2_area) - inter_area
  
  return inter_area/union_area

In [5]:
box1 = (2, 1, 4, 3)
box2 = (1, 2, 3, 4) 
print("iou = " + str(iou(box1, box2)))

iou = 0.14285714285714285


In [0]:
def yolo_non_max_suppression(scores, boxes, classes, max_boxes = 10, iou_threshold = 0.5):
  max_boxes_tensor = K.variable(max_boxes, dtype = "int32")
  K.get_session().run(tf.variables_initializer([max_boxes_tensor]))
  nms_indices = tf.image.non_max_suppression(boxes, scores, max_boxes_tensor, iou_threshold = iou_threshold)
  scores = K.gather(scores, nms_indices)
  boxes = K.gather(boxes, nms_indices)
  classes = K.gather(classes, nms_indices)
  return scores, boxes, classes

In [0]:
def scale_boxes(boxes, image_shape):
    """ Scales the predicted boxes in order to be drawable on the image"""
    height = image_shape[0]
    width = image_shape[1]
    image_dims = K.stack([height, width, height, width])
    image_dims = K.reshape(image_dims, [1, 4])
    boxes = boxes * image_dims
    return boxes

In [0]:
def yolo_boxes_to_corners(box_xy, box_wh):
    """Convert YOLO box predictions to bounding box corners."""
    box_mins = box_xy - (box_wh / 2.)
    box_maxes = box_xy + (box_wh / 2.)

    return K.concatenate([
        box_mins[..., 1:2],  # y_min
        box_mins[..., 0:1],  # x_min
        box_maxes[..., 1:2],  # y_max
        box_maxes[..., 0:1]  # x_max
    ])


In [0]:
def yolo_eval(yolo_outputs, image_shape = (720., 1280.), max_boxes=10, score_threshold=.6, iou_threshold=.5):
  box_confidence, box_xy, box_wh, box_class_probs = yolo_outputs
  boxes = yolo_boxes_to_corners(box_xy, box_wh)
  scores, boxes, classes = yolo_filter_boxes(box_confidence, boxes, box_class_probs, threshold = score_threshold)
  boxes = scale_boxes(boxes,image_shape)
  scores, boxes, classes = yolo_non_max_suppression(scores, boxes, classes, max_boxes = max_boxes, iou_threshold = iou_threshold)
    
    
    
  return scores, boxes, classes

In [12]:
with tf.Session() as test_b:
    yolo_outputs = (tf.random_normal([19, 19, 5, 1], mean=1, stddev=4, seed = 1),
                    tf.random_normal([19, 19, 5, 2], mean=1, stddev=4, seed = 1),
                    tf.random_normal([19, 19, 5, 2], mean=1, stddev=4, seed = 1),
                    tf.random_normal([19, 19, 5, 80], mean=1, stddev=4, seed = 1))
    scores, boxes, classes = yolo_eval(yolo_outputs)
    print("scores[2] = " + str(scores[2].eval()))
    print("boxes[2] = " + str(boxes[2].eval()))
    print("classes[2] = " + str(classes[2].eval()))
    print("scores.shape = " + str(scores.eval().shape))
    print("boxes.shape = " + str(boxes.eval().shape))
    print("classes.shape = " + str(classes.eval().shape))

W0622 15:41:49.221451 139694429996928 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0622 15:41:49.222419 139694429996928 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:190: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.

W0622 15:41:49.225008 139694429996928 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:199: The name tf.is_variable_initialized is deprecated. Please use tf.compat.v1.is_variable_initialized instead.

W0622 15:41:49.254217 139694429996928 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:206: The name tf.variables_initializer is deprecated. Please use tf.compat.v1.variables_initiali

(19, 19, 5, 80)
(19, 19, 5)
(19, 19, 5)
(19, 19, 5)
scores[2] = 138.79124
boxes[2] = [1292.3297  -278.52167 3876.9893  -835.56494]
classes[2] = 54
scores.shape = (10,)
boxes.shape = (10, 4)
classes.shape = (10,)


In [13]:
sess = K.get_session()

W0622 15:47:04.016402 139694429996928 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

